In [175]:
# scales - SR
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [176]:
import pandas as pd
import numpy as np
import os

In [177]:
df_labelled = pd.read_csv('./YAD/Labelled+webmini.csv')
print(df_labelled)
print(type(df_labelled))

     Unnamed: 0        ID  Gender  Age  PHQ9_total  GAD7_total STAI_X1_total  \
0             0  YAD30211       1   27          14           9            47   
1             1  YAD30504       1   24           3           3            33   
2             2  YAD30211       1   26          22          12            47   
3             3  YAD30212       1   24          22          10            29   
4             4  YAD30213       1   28           8           3            36   
..          ...       ...     ...  ...         ...         ...           ...   
289         289  MDD20019       1   24          10           8            64   
290         290  MDD20020       0   25          10          14            71   
291         291  MDD20021       0   31          17          12            62   
292         292  MDD20022       0   20          14          11         empty   
293         293  MDD20023       1   25           4           0            36   

    LSAS_performance_fear LSAS_situatio

In [178]:
df_unlabelled_K = pd.read_csv('./YAD/Unlabelled_KAIST.csv')
print(df_unlabelled_K)

          No                                        ID  \
0        494  438550c7ec70a9ba62a114e93586f6c4f2490cb2   
1        501  09f679c6c3be3da2ba3a6b1f09cfc582a1fd138b   
2        505  a73b7ab092df5a28b7ea79e6803a876332fe13dc   
3        510  72f26d01617b27d594f3c31647d88732014c010c   
4        511  38857c6812e0f37b74db69a9f0910cb3a5b933a2   
...      ...                                       ...   
17247  30490  840842512161278f5da5628a4f4accd5834fa1ed   
17248  30491  c474c86376cd13ba738e36dbf58a09789666ed0b   
17249  30492  7819ea09468d5e7d78746728bee2392c70c5ba59   
17250  30494  56a4001930985c21ef45a8436a77c6e55833f0b6   
17251  30497  f3dc642ddf1b0ab1030d08c19a188086a24bc01a   

                                           ID_2 appointment_date  Age  Gender  \
0      54efb2832e5ca9e8b299ce827c112a1c965c14e2       2017-03-24   24       1   
1      4b012a3cb9d1866e62bf12c018a05e7896ff5b80       2017-03-24   29       1   
2      139991fcc7a5504432014eebaaed80479fb37682       2017-0

C:\Users\CNDLMembers\anaconda3\envs\pytorch\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (92) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [179]:
print(df_unlabelled_K.columns)

Index(['No', 'ID', 'ID_2', 'appointment_date', 'Age', 'Gender', 'Grade',
       'SMOKING_01', 'SMOKING_cigar', 'SMOKING_yrs',
       ...
       'KSSI_1y_11', 'KSSI_1y_12', 'KSSI_1y_13', 'KSSI_1y_14', 'KSSI_1y_Total',
       'KSSI_atpt_1y', 'KSSI_atpt_life', 'KSSI_atpt_life_no',
       'KSSI_atpt_life_plan', 'KSSI_atpt_life_impul'],
      dtype='object', length=291)


In [180]:
print(df_labelled['PHQ9_total'].dtypes)
print(df_labelled['STAI_X1_total'].dtypes)

int64
object


In [181]:
df_labelled['STAI_X1_total'] = pd.to_numeric(df_labelled['STAI_X1_total'], errors='coerce') 

print(df_labelled['STAI_X1_total'].dtypes)

float64


In [182]:
train_df_labelled = df_labelled[df_labelled['site'].isin(['KAIST', 'Gachon'])]
test_df_labelled = df_labelled[df_labelled['site'] == 'SMC']
valid_df_labelled = df_labelled[df_labelled['site'] == 'SNU']

In [183]:
train_x = train_df_labelled.loc[:, ['PHQ9_total', 'GAD7_total', 'STAI_X1_total']]
test_x = test_df_labelled.loc[:, ['PHQ9_total', 'GAD7_total', 'STAI_X1_total']]
valid_x = valid_df_labelled.loc[:, ['PHQ9_total', 'GAD7_total', 'STAI_X1_total']]

print(train_x)

     PHQ9_total  GAD7_total  STAI_X1_total
0            14           9           47.0
1             3           3           33.0
2            22          12           47.0
3            22          10           29.0
4             8           3           36.0
..          ...         ...            ...
134           2           5           45.0
135           9           2           29.0
136           4           2           42.0
137          15          14           65.0
138           6           2           44.0

[139 rows x 3 columns]


In [184]:
train_y = train_df_labelled.loc[:, ['KSSI_total', 'suicidal_idea', 'suicidal_plan', 'suicidal_attempt']]
test_y = test_df_labelled.loc[:, ['KSSI_total', 'suicidal_idea', 'suicidal_plan', 'suicidal_attempt']]
valid_y = valid_df_labelled.loc[:, ['KSSI_total', 'suicidal_idea', 'suicidal_plan', 'suicidal_attempt']]

print(train_y)
print(type(train_y))
print(train_df_labelled['KSSI_total'].dtype)
print(train_df_labelled['suicidal_idea'].dtype)
print(train_df_labelled['suicidal_plan'].dtype)
print(train_df_labelled['suicidal_attempt'].dtype)

train_y = train_y.apply(pd.to_numeric, errors='coerce')
test_y = test_y.apply(pd.to_numeric, errors='coerce')
valid_y = valid_y.apply(pd.to_numeric, errors='coerce')

print(train_y['KSSI_total'].dtype)
print(train_y['suicidal_idea'].dtype)
print(train_y['suicidal_plan'].dtype)
print(train_y['suicidal_attempt'].dtype)

train_y['sum'] = train_y['KSSI_total'] + train_y['suicidal_idea'] + train_y['suicidal_plan'] + train_y['suicidal_attempt']
train_y = train_y.iloc[:, 4]
#train_y = train_y.drop(['KSSI_total', 'suicidal_idea', 'suicidal_plan', 'suicidal_attempt'], axis=1)
print(train_y)
print(type(train_y))

    KSSI_total suicidal_idea suicidal_plan suicidal_attempt
0           12             1             0                0
1            1             0             0                0
2           25             1             0                0
3           50             1             1                1
4            1             0             0                0
..         ...           ...           ...              ...
134          0             0             0                0
135          0             0             0                0
136          1             0             0                0
137         44             1             1                0
138          3             0             0                0

[139 rows x 4 columns]
<class 'pandas.core.frame.DataFrame'>
object
object
object
object
float64
int64
int64
int64
0      13.0
1       1.0
2      26.0
3      53.0
4       1.0
       ... 
134     0.0
135     0.0
136     1.0
137    46.0
138     3.0
Name: sum, Length: 139, dtype: f

In [185]:
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [186]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


In [187]:
x_train = torch.from_numpy(train_x.values)
print(x_train.shape)
print(x_train.type())
y_train = torch.from_numpy(train_y.values)
torch.nan_to_num_(x_train)
print(y_train.shape)
print(y_train.unsqueeze_(1))
print(y_train.shape)
torch.nan_to_num_(y_train)
print(y_train)

torch.Size([139, 3])
torch.DoubleTensor
torch.Size([139])
tensor([[ 13.],
        [  1.],
        [ 26.],
        [ 53.],
        [  1.],
        [ 39.],
        [ 54.],
        [ nan],
        [ 70.],
        [ 97.],
        [ 52.],
        [ 77.],
        [ 10.],
        [  5.],
        [ 41.],
        [ 36.],
        [ 27.],
        [107.],
        [ 14.],
        [ 93.],
        [ 23.],
        [ 24.],
        [ 20.],
        [ 38.],
        [ 51.],
        [  6.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  4.],
        [  0.],
        [  0.],
        [  0.],
        [ 13.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  2.],
        [ 45.],
        [ 19.],
        [  4.],
        [ 43.],
        [ 32.],
        [ 67.],
        [ 59.],
        [ 18.],
        [ 83.],
        [ 26.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.]

In [188]:
W = torch.zeros((3, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
hypothesis = x_train.float().matmul(W) + b 
print(hypothesis)
# model = nn.Linear(3, 1)
# print(hypothesis.shape)

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
      

In [189]:
# cost = torch.mean((hypothesis - y_train.float())**2)
# print(cost.shape)

In [190]:
optimizer = torch.optim.SGD([W, b], lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50, eta_min=0)

In [191]:
nb_epochs = 10000
for epoch in range(1, nb_epochs + 1):
    hypothesis = x_train.float().matmul(W) + b
    cost = torch.mean((hypothesis - y_train.float())**2)   
# torch.mean을 해주면 최종 값의 size가 torch.Size([])이 됨. 따라서 hypothesis(x_train으로 만든)나 y_train 값중 하나라고 nan값이 있으면 cost도 nan 됨
 
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
  
    if(epoch%100==0):
        print('Epochs {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
            epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()
        ))

Epochs  100/10000 hypothesis: tensor([26.2151,  6.4332, 39.4670, 37.8211, 13.0521, 40.3487, 47.4437,  7.2925,
        35.5111, 30.7624, 29.3542, 37.3369, 13.1410, 14.3937, 39.0717, 21.0772,
        22.3278, 40.3870, 12.6487, 46.5177, 36.0095, 28.5696, 31.0648, 24.8412,
        31.1274, 26.4051,  1.0972,  3.7723, -0.2484, -0.2181, -0.2484,  2.4428,
        -0.2666,  1.9748, -0.2362,  0.5988,  1.1336,  6.4311,  2.3882, -0.2726,
         0.6534, -0.2241, -0.3030,  5.1098, 34.6982, 28.4888, 24.6014, 42.9792,
        17.0565, 26.2535, 23.7381, 12.6386, 33.4355, 32.4286, -0.2302,  5.9691,
         2.4246, -0.2120, -0.2302,  1.0790, -0.2726,  4.6963,  2.4610,  1.0972,
         1.5128,  2.3943, -0.2241, -0.2848,  0.5988, 18.1899, 11.2020,  3.7824,
         5.5071, 10.2659,  2.4671,  2.4732, -0.1635,  6.3887, 14.2724,  1.5552,
         2.0294, 16.9555, 32.2204, 19.5295, 29.2652,  8.9788, 13.8912, 24.3933,
        24.3711, 10.7865,  2.0415, 23.9070,  1.0912,  2.0172,  5.9348,  8.9667,
         5

Epochs  600/10000 hypothesis: tensor([ 3.0660e+01,  5.1674e+00,  4.8863e+01,  4.8726e+01,  1.4103e+01,
         5.0176e+01,  6.0418e+01,  5.9340e+00,  4.3856e+01,  3.9183e+01,
         3.5894e+01,  4.7386e+01,  1.5750e+01,  1.6292e+01,  4.9169e+01,
         2.6409e+01,  2.6852e+01,  5.0793e+01,  1.4194e+01,  5.8371e+01,
         4.5529e+01,  3.6443e+01,  3.7212e+01,  2.7970e+01,  3.8295e+01,
         3.4187e+01, -2.8249e+00,  1.3370e+00, -5.0310e+00, -4.4491e+00,
        -5.0310e+00, -6.1869e-01, -5.3802e+00, -1.7091e+00, -4.7983e+00,
        -4.4972e+00, -2.1266e+00,  5.0687e+00, -1.6661e+00, -5.4966e+00,
        -3.4498e+00, -4.5655e+00, -6.0785e+00,  3.3281e+00,  4.3922e+01,
         3.5011e+01,  3.1024e+01,  5.3425e+01,  2.0041e+01,  3.1457e+01,
         3.0240e+01,  1.4060e+01,  4.3246e+01,  3.9768e+01, -4.6819e+00,
         4.0947e+00, -9.6785e-01, -4.3327e+00, -4.6819e+00, -3.1740e+00,
        -5.4966e+00,  3.2851e+00, -2.6954e-01, -2.8249e+00, -2.6832e+00,
        -1.5498e+00, 

Epochs 1000/10000 hypothesis: tensor([ 3.0673e+01,  5.1377e+00,  4.8975e+01,  4.8941e+01,  1.4199e+01,
         4.9941e+01,  6.0222e+01,  5.8628e+00,  4.3741e+01,  3.9369e+01,
         3.5720e+01,  4.7373e+01,  1.6025e+01,  1.6261e+01,  4.9233e+01,
         2.6677e+01,  2.6969e+01,  5.0880e+01,  1.4395e+01,  5.8198e+01,
         4.5490e+01,  3.6643e+01,  3.7109e+01,  2.7790e+01,  3.8520e+01,
         3.4563e+01, -2.8100e+00,  1.2537e+00, -5.0471e+00, -4.4570e+00,
        -5.0471e+00, -5.7293e-01, -5.4012e+00, -1.7309e+00, -4.8111e+00,
        -4.5581e+00, -2.1018e+00,  5.1941e+00, -1.6352e+00, -5.5192e+00,
        -3.4958e+00, -4.5750e+00, -6.1094e+00,  3.4291e+00,  4.4017e+01,
         3.4877e+01,  3.1460e+01,  5.3235e+01,  2.0376e+01,  3.1325e+01,
         3.0561e+01,  1.4103e+01,  4.3489e+01,  3.9700e+01, -4.6930e+00,
         4.1542e+00, -9.2702e-01, -4.3389e+00, -4.6930e+00, -3.1641e+00,
        -5.5192e+00,  3.3335e+00, -2.1884e-01, -2.8100e+00, -2.7708e+00,
        -1.5172e+00, 

Epochs 1400/10000 hypothesis: tensor([ 3.0674e+01,  5.1267e+00,  4.9027e+01,  4.9061e+01,  1.4255e+01,
         4.9800e+01,  6.0103e+01,  5.8265e+00,  4.3670e+01,  3.9475e+01,
         3.5620e+01,  4.7362e+01,  1.6190e+01,  1.6249e+01,  4.9263e+01,
         2.6834e+01,  2.7036e+01,  5.0921e+01,  1.4514e+01,  5.8089e+01,
         4.5465e+01,  3.6758e+01,  3.7043e+01,  2.7678e+01,  3.8642e+01,
         3.4782e+01, -2.7977e+00,  1.2125e+00, -5.0534e+00, -4.4562e+00,
        -5.0534e+00, -5.4200e-01, -5.4117e+00, -1.7396e+00, -4.8145e+00,
        -4.5925e+00, -2.0811e+00,  5.2696e+00, -1.6169e+00, -5.5311e+00,
        -3.5176e+00, -4.5756e+00, -6.1283e+00,  3.4916e+00,  4.4069e+01,
         3.4801e+01,  3.1710e+01,  5.3117e+01,  2.0568e+01,  3.1248e+01,
         3.0748e+01,  1.4133e+01,  4.3628e+01,  3.9657e+01, -4.6951e+00,
         4.1914e+00, -9.0031e-01, -4.3368e+00, -4.6951e+00, -3.1560e+00,
        -5.5311e+00,  3.3689e+00, -1.8370e-01, -2.7977e+00, -2.8178e+00,
        -1.4975e+00, 

Epochs 1900/10000 hypothesis: tensor([ 3.0670e+01,  5.1266e+00,  4.9056e+01,  4.9147e+01,  1.4296e+01,
         4.9709e+01,  6.0029e+01,  5.8076e+00,  4.3625e+01,  3.9553e+01,
         3.5559e+01,  4.7363e+01,  1.6309e+01,  1.6249e+01,  4.9284e+01,
         2.6947e+01,  2.7084e+01,  5.0948e+01,  1.4599e+01,  5.8016e+01,
         4.5456e+01,  3.6845e+01,  3.6996e+01,  2.7599e+01,  3.8720e+01,
         3.4937e+01, -2.7885e+00,  1.1922e+00, -5.0585e+00, -4.4529e+00,
        -5.0585e+00, -5.1854e-01, -5.4218e+00, -1.7442e+00, -4.8163e+00,
        -4.6197e+00, -2.0619e+00,  5.3225e+00, -1.6085e+00, -5.5429e+00,
        -3.5297e+00, -4.5740e+00, -6.1484e+00,  3.5370e+00,  4.4109e+01,
         3.4757e+01,  3.1880e+01,  5.3038e+01,  2.0697e+01,  3.1201e+01,
         3.0882e+01,  1.4161e+01,  4.3730e+01,  3.9629e+01, -4.6952e+00,
         4.2180e+00, -8.8186e-01, -4.3318e+00, -4.6952e+00, -3.1518e+00,
        -5.5429e+00,  3.4013e+00, -1.5521e-01, -2.7885e+00, -2.8487e+00,
        -1.4874e+00, 

...

Epochs 7500/10000 hypothesis: tensor([ 3.0561e+01,  5.2914e+00,  4.8919e+01,  4.9314e+01,  1.4384e+01,
         4.9759e+01,  6.0155e+01,  5.9244e+00,  4.3664e+01,  3.9750e+01,
         3.5702e+01,  4.7527e+01,  1.6565e+01,  1.6458e+01,  4.9312e+01,
         2.7178e+01,  2.7176e+01,  5.0937e+01,  1.4742e+01,  5.7999e+01,
         4.5625e+01,  3.7077e+01,  3.6933e+01,  2.7448e+01,  3.8664e+01,
         3.5200e+01, -2.7649e+00,  1.3497e+00, -5.0818e+00, -4.3895e+00,
        -5.0818e+00, -4.4789e-01, -5.4972e+00, -1.7164e+00, -4.8049e+00,
        -4.7257e+00, -1.9341e+00,  5.3953e+00, -1.6941e+00, -5.6357e+00,
        -3.4795e+00, -4.5280e+00, -6.3280e+00,  3.6322e+00,  4.4243e+01,
         3.4930e+01,  3.1995e+01,  5.3009e+01,  2.0746e+01,  3.1288e+01,
         3.1120e+01,  1.4362e+01,  4.3968e+01,  3.9666e+01, -4.6664e+00,
         4.2653e+00, -8.6328e-01, -4.2511e+00, -4.6664e+00, -3.1802e+00,
        -5.6357e+00,  3.6098e+00, -3.2498e-02, -2.7649e+00, -2.8465e+00,
        -1.5556e+00, 

Epochs 8000/10000 hypothesis: tensor([ 3.0551e+01,  5.3072e+00,  4.8903e+01,  4.9322e+01,  1.4388e+01,
         4.9774e+01,  6.0175e+01,  5.9376e+00,  4.3672e+01,  3.9761e+01,
         3.5721e+01,  4.7542e+01,  1.6577e+01,  1.6478e+01,  4.9313e+01,
         2.7189e+01,  2.7180e+01,  5.0933e+01,  1.4748e+01,  5.8005e+01,
         4.5642e+01,  3.7090e+01,  3.6932e+01,  2.7441e+01,  3.8651e+01,
         3.5209e+01, -2.7635e+00,  1.3669e+00, -5.0836e+00, -4.3838e+00,
        -5.0836e+00, -4.4348e-01, -5.5035e+00, -1.7133e+00, -4.8036e+00,
        -4.7331e+00, -1.9237e+00,  5.3970e+00, -1.7031e+00, -5.6434e+00,
        -3.4735e+00, -4.5237e+00, -6.3432e+00,  3.6368e+00,  4.4252e+01,
         3.4951e+01,  3.1989e+01,  5.3014e+01,  2.0738e+01,  3.1301e+01,
         3.1129e+01,  1.4378e+01,  4.3981e+01,  3.9672e+01, -4.6637e+00,
         4.2671e+00, -8.6336e-01, -4.2438e+00, -4.6637e+00, -3.1834e+00,
        -5.6434e+00,  3.6266e+00, -2.3589e-02, -2.7635e+00, -2.8431e+00,
        -1.5632e+00, 

Epochs 8500/10000 hypothesis: tensor([ 3.0541e+01,  5.3230e+00,  4.8887e+01,  4.9329e+01,  1.4392e+01,
         4.9788e+01,  6.0194e+01,  5.9507e+00,  4.3681e+01,  3.9771e+01,
         3.5741e+01,  4.7558e+01,  1.6590e+01,  1.6498e+01,  4.9313e+01,
         2.7199e+01,  2.7184e+01,  5.0929e+01,  1.4753e+01,  5.8010e+01,
         4.5659e+01,  3.7104e+01,  3.6931e+01,  2.7435e+01,  3.8638e+01,
         3.5219e+01, -2.7622e+00,  1.3841e+00, -5.0853e+00, -4.3780e+00,
        -5.0853e+00, -4.3910e-01, -5.5097e+00, -1.7102e+00, -4.8024e+00,
        -4.7405e+00, -1.9135e+00,  5.3986e+00, -1.7122e+00, -5.6511e+00,
        -3.4675e+00, -4.5195e+00, -6.3584e+00,  3.6413e+00,  4.4260e+01,
         3.4972e+01,  3.1983e+01,  5.3019e+01,  2.0730e+01,  3.1314e+01,
         3.1138e+01,  1.4394e+01,  4.3993e+01,  3.9678e+01, -4.6609e+00,
         4.2690e+00, -8.6346e-01, -4.2366e+00, -4.6609e+00, -3.1866e+00,
        -5.6511e+00,  3.6433e+00, -1.4731e-02, -2.7622e+00, -2.8398e+00,
        -1.5707e+00, 

Epochs 8900/10000 hypothesis: tensor([ 3.0533e+01,  5.3356e+00,  4.8874e+01,  4.9335e+01,  1.4396e+01,
         4.9799e+01,  6.0209e+01,  5.9611e+00,  4.3688e+01,  3.9780e+01,
         3.5757e+01,  4.7570e+01,  1.6600e+01,  1.6514e+01,  4.9314e+01,
         2.7208e+01,  2.7187e+01,  5.0926e+01,  1.4757e+01,  5.8015e+01,
         4.5673e+01,  3.7114e+01,  3.6930e+01,  2.7430e+01,  3.8627e+01,
         3.5226e+01, -2.7611e+00,  1.3977e+00, -5.0867e+00, -4.3735e+00,
        -5.0867e+00, -4.3560e-01, -5.5146e+00, -1.7077e+00, -4.8014e+00,
        -4.7464e+00, -1.9053e+00,  5.3999e+00, -1.7194e+00, -5.6573e+00,
        -3.4627e+00, -4.5161e+00, -6.3705e+00,  3.6449e+00,  4.4267e+01,
         3.4989e+01,  3.1978e+01,  5.3024e+01,  2.0723e+01,  3.1324e+01,
         3.1146e+01,  1.4407e+01,  4.4003e+01,  3.9683e+01, -4.6587e+00,
         4.2704e+00, -8.6353e-01, -4.2308e+00, -4.6587e+00, -3.1891e+00,
        -5.6573e+00,  3.6566e+00, -7.6684e-03, -2.7611e+00, -2.8371e+00,
        -1.5767e+00, 

Epochs 9500/10000 hypothesis: tensor([ 3.0521e+01,  5.3543e+00,  4.8855e+01,  4.9344e+01,  1.4401e+01,
         4.9816e+01,  6.0233e+01,  5.9766e+00,  4.3697e+01,  3.9793e+01,
         3.5780e+01,  4.7589e+01,  1.6615e+01,  1.6538e+01,  4.9314e+01,
         2.7220e+01,  2.7191e+01,  5.0922e+01,  1.4763e+01,  5.8022e+01,
         4.5693e+01,  3.7130e+01,  3.6928e+01,  2.7422e+01,  3.8612e+01,
         3.5237e+01, -2.7595e+00,  1.4181e+00, -5.0887e+00, -4.3666e+00,
        -5.0887e+00, -4.3036e-01, -5.5220e+00, -1.7040e+00, -4.7999e+00,
        -4.7553e+00, -1.8930e+00,  5.4018e+00, -1.7301e+00, -5.6664e+00,
        -3.4555e+00, -4.5111e+00, -6.3885e+00,  3.6503e+00,  4.4278e+01,
         3.5014e+01,  3.1971e+01,  5.3030e+01,  2.0714e+01,  3.1340e+01,
         3.1157e+01,  1.4427e+01,  4.4018e+01,  3.9690e+01, -4.6555e+00,
         4.2726e+00, -8.6362e-01, -4.2222e+00, -4.6555e+00, -3.1928e+00,
        -5.6664e+00,  3.6765e+00,  2.8999e-03, -2.7595e+00, -2.8332e+00,
        -1.5857e+00, 

In [205]:
import torch
from torch.autograd import Variable

class PModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = torch.nn.Linear(3, 1)
        
    def forward(self, x):
        return self.linear(x)
        
model = PModel()

In [206]:
optimizer = torch.optim.SGD(model.parameters(),lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50, eta_min=0)

In [207]:
nb_epochs =10000
for epoch in range(1, nb_epochs+1):
    prediction = model(x_train.float())
    cost = F.mse_loss(prediction.squeeze(), y_train.float())

    optimizer.zero_grad() 
    cost.backward() 
    optimizer.step() 
    
    if(epoch%100==0):
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
          epoch, nb_epochs, cost.item()
      ))

<ipython-input-207-5fbc32b2f35d>:4: UserWarning: Using a target size (torch.Size([139, 1])) that is different to the input size (torch.Size([139])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  cost = F.mse_loss(prediction.squeeze(), y_train.float())


Epoch  100/10000 Cost: 502.843536
Epoch  200/10000 Cost: 502.676056
Epoch  300/10000 Cost: 502.602081
Epoch  400/10000 Cost: 502.546600
Epoch  500/10000 Cost: 502.501526
Epoch  600/10000 Cost: 502.464111
Epoch  700/10000 Cost: 502.432526
Epoch  800/10000 Cost: 502.405457
Epoch  900/10000 Cost: 502.381927
Epoch 1000/10000 Cost: 502.360962
Epoch 1100/10000 Cost: 502.342010
Epoch 1200/10000 Cost: 502.324738
Epoch 1300/10000 Cost: 502.308533
Epoch 1400/10000 Cost: 502.293365
Epoch 1500/10000 Cost: 502.278870
Epoch 1600/10000 Cost: 502.264893
Epoch 1700/10000 Cost: 502.251343
Epoch 1800/10000 Cost: 502.238129
Epoch 1900/10000 Cost: 502.225250
Epoch 2000/10000 Cost: 502.212463
Epoch 2100/10000 Cost: 502.199890
Epoch 2200/10000 Cost: 502.187469
Epoch 2300/10000 Cost: 502.175140
Epoch 2400/10000 Cost: 502.162933
Epoch 2500/10000 Cost: 502.150726
Epoch 2600/10000 Cost: 502.138611
Epoch 2700/10000 Cost: 502.126587
Epoch 2800/10000 Cost: 502.114594
Epoch 2900/10000 Cost: 502.102570
Epoch 3000/100

In [196]:
print(list(model.parameters()))

[Parameter containing:
tensor([[ 0.3392, -0.4443,  0.2725]], requires_grad=True), Parameter containing:
tensor([0.4773], requires_grad=True)]


In [197]:
###Test
x_test = torch.from_numpy(test_x.values)
print(x_test.shape)
print(x_test.type())
y_test = torch.from_numpy(test_y.values)
print(y_test.shape)
print(y_test.type())


torch.Size([91, 3])
torch.DoubleTensor
torch.Size([91, 4])
torch.DoubleTensor


In [198]:
model_t = PModel()

criterion = torch.nn.MSELoss(size_average=False) 
optimizer = torch.optim.SGD(model.parameters(),lr=0.01)

for epoch in range(501):
    y_pred =model(x_test.float())
    loss = criterion(y_pred, y_test.float()) 
    
    optimizer.zero_grad() 
    loss.backward() 
    optimizer.step() 
    
    if(epoch%10==0):
        print(epoch, loss.item())
    
    

0 nan
10 nan
20 nan
30 nan
40 nan
50 nan
60 nan
70 nan
80 nan
90 nan
100 nan
110 nan
120 nan
130 nan
140 nan
150 nan
160 nan
170 nan
180 nan
190 nan
200 nan
210 nan
220 nan
230 nan
240 nan
250 nan
260 nan
270 nan
280 nan
290 nan
300 nan
310 nan
320 nan
330 nan
340 nan
350 nan
360 nan
370 nan
380 nan
390 nan
400 nan
410 nan
420 nan
430 nan
440 nan
450 nan
460 nan
470 nan
480 nan
490 nan
500 nan


C:\Users\CNDLMembers\anaconda3\envs\pytorch\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
C:\Users\CNDLMembers\anaconda3\envs\pytorch\lib\site-packages\torch\nn\modules\loss.py:528: UserWarning: Using a target size (torch.Size([91, 4])) that is different to the input size (torch.Size([91, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
